<a href="https://colab.research.google.com/github/motasem00/data-driven-modelling/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import bibs
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from matplotlib.patches import Patch
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import TruncatedSVD

In [ ]:
%cd /content/

/content


In [ ]:
!ls

data-driven-modelling  sample_data


In [ ]:
repo_dir = 'data-driven-modelling'
if os.path.exists(repo_dir):
    !rm -rf {repo_dir}

In [2]:
repo_url = 'https://github.com/motasem00/data-driven-modelling.git'
!git clone {repo_url}
%cd data-driven-modelling

Cloning into 'data-driven-modelling'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 96 (delta 45), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (96/96), 36.75 MiB | 6.20 MiB/s, done.
Resolving deltas: 100% (45/45), done.
/content/data-driven-modelling


# Loading data

In [3]:
%cd /content/data-driven-modelling/notebooks//

/content/data-driven-modelling/notebooks


In [4]:
working_directory = os.getcwd()
path = working_directory
path

'/content/data-driven-modelling/notebooks'

In [5]:
df_gdp = pd.read_pickle("/content/data-driven-modelling/data/raw/gdp.pickle")
df_pattern = pd.read_pickle("/content/data-driven-modelling/data/raw/naics_pattern.pickle")

df_occupation_1 = pd.read_pickle("/content/data-driven-modelling/data/raw/naics_occupation_part1.pickle")
df_occupation_2 = pd.read_pickle("/content/data-driven-modelling/data/raw/naics_occupation_part2.pickle")
df_occupation_3 = pd.read_pickle("/content/data-driven-modelling/data/raw/naics_occupation_part3.pickle")

In [6]:
county_data = pd.read_pickle("/content/data-driven-modelling/data/raw/county.pickle")
State_data = pd.read_pickle("/content/data-driven-modelling/data/raw/state.pickle")

In [7]:
df_pattern['FIPS'] = df_pattern['FIPS'].astype(str)
unique_lengths = df_pattern['FIPS'].apply(len).unique()
unique_lengths

array([4, 5])

In [8]:
def add_zeros(code):
    if len(code) == 3:
        return '00' + code
    elif len(code) == 4:
        return '0' + code
    elif len(code) == 1:
        return '0000' + code
    return code

In [9]:
df_pattern['FIPS'] = df_pattern['FIPS'].apply(add_zeros)

In [10]:
df_occupation = pd.concat([df_occupation_1, df_occupation_2, df_occupation_3], ignore_index=True)
df_occupation.head(2)

,FIPS,State_GEOID,naics,NAICS_TITLE,emp_total_county_naics,OCC_CODE,OCC_TITLE,emp_occupation,state_name
0,12999,12,5613,Employment Services,1436559,49-9071,"Maintenance and Repair Workers, General",20639.514235,NaN
1,06999,6,5613,Employment Services,729335,49-9071,"Maintenance and Repair Workers, General",9414.167765,NaN


# My project plan will be as following:

* Upload and Link the datasets.
* And I will try to understand the data and find a way of matching things and find a working model.
* Then I will try to clean the data and drop the un-necessary columns.



### We will start with the GDP Data

In [11]:
df_gdp.columns

Index(['FIPS', 'GeoName', 'Region', 'TableName', 'LineCode',
       'IndustryClassification', 'Description', 'Unit', '2017', '2018', '2019',
       '2020', '2021', '2022'],
      dtype='object')

In [12]:
columns_to_drop_gdp = [
    'GeoName', 'Region', 'TableName', 'LineCode','Unit'
]

# Revomve not needed columns
filtered_gdp = df_gdp.drop(columns=columns_to_drop_gdp, errors='ignore')

# Show the rest
print(filtered_gdp.columns)

Index(['FIPS', 'IndustryClassification', 'Description', '2017', '2018', '2019',
       '2020', '2021', '2022'],
      dtype='object')


In [64]:
filtered_gdp

,FIPS,IndustryClassification,Description,2017,2018,2019,2020,2021,2022
0,0,...,All industry total,1.961210e+10,2.019390e+10,2.069209e+10,2.023407e+10,2.140769e+10,2.182204e+10
1,0,...,Private industries,1.715626e+10,1.771178e+10,1.819575e+10,1.776187e+10,1.890945e+10,1.928312e+10
2,0,11,"Agriculture, forestry, fishing and hunting",1.768400e+08,1.841050e+08,1.711490e+08,1.736590e+08,1.837420e+08,1.700580e+08
3,0,21,"Mining, quarrying, and oil and gas extraction",2.673020e+08,2.770130e+08,3.146040e+08,3.055880e+08,2.694780e+08,2.452900e+08
4,0,22,Utilities,3.137110e+08,3.092690e+08,3.129200e+08,3.323360e+08,3.166030e+08,3.183920e+08
...,...,...,...,...,...,...,...,...,...
108047,98000,"42,44-45",Trade,4.461445e+08,4.576763e+08,4.741341e+08,4.674147e+08,4.661462e+08,4.292295e+08
108048,98000,"22,48-49",Transportation and utilities,1.674685e+08,1.733558e+08,1.814022e+08,1.711564e+08,1.857088e+08,1.903768e+08
108049,98000,"31-33,51",Manufacturing and information,7.469147e+08,7.976669e+08,8.595105e+08,8.771524e+08,9.864447e+08,1.010181e+09
108050,98000,...,Private goods-producing industries 2/,6.350413e+08,6.706487e+08,6.898229e+08,6.691179e+08,6.954938e+08,6.650949e+08


### Then I Will have a look at the pattern dataset

In [13]:
df_pattern.columns

Index(['State_GEOID', 'County_GEOID', 'FIPS', 'naics_2', 'naics',
       'DESCRIPTION', 'emp_nf', 'emp', 'qp1_nf', 'qp1', 'ap_nf', 'ap', 'est',
       'n<5', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499',
       'n500_999', 'n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4'],
      dtype='object')

In [14]:
columns_to_drop_pattern = [
    'State_GEOID', 'County_GEOID','naics_2', 'emp_nf', 'qp1_nf', 'qp1', 'ap_nf',
    'n<5', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499',
       'n500_999', 'n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4'
]

# Revomve not needed columns
filtered_pattern = df_pattern.drop(columns=columns_to_drop_pattern, errors='ignore')

# Show the rest
print(filtered_pattern.columns)

Index(['FIPS', 'naics', 'DESCRIPTION', 'emp', 'ap', 'est'], dtype='object')


In [15]:
filtered_pattern = filtered_pattern.astype({"FIPS": int})

### Let's now have a look at the Occupation Dataset

In [49]:
columns_to_drop_occupation = [
    'State_GEOID',  'emp_total_county_naics', 'state_name'
]

# Revomve not needed columns
df_filtered_occupation = df_occupation.drop(columns=columns_to_drop_occupation, errors='ignore')

# Show the rest
print(df_filtered_occupation.columns)

Index(['FIPS', 'naics', 'NAICS_TITLE', 'OCC_CODE', 'OCC_TITLE',
       'emp_occupation'],
      dtype='object')


In [50]:
df_filtered_occupation.rename(columns={'GEOID': 'FIPS'}, inplace=True)

In [51]:
df_filtered_occupation = df_filtered_occupation.astype({"FIPS": int})

### We Got a problem

---


*When we did the ranking for industries we found some NAICS are in 6 digits and that's why we had to create a function to expand those NAICS to it's representative 4 digit Industries codes*

In [19]:
pattern_data = filtered_pattern

In [20]:
print(pattern_data.columns)

Index(['FIPS', 'naics', 'DESCRIPTION', 'emp', 'ap', 'est'], dtype='object')


In [21]:
naics_expanding = {
    '3330A1': {
        'codes': ['3331', '3332', '3334', '3339'],
        'description': [
            'Agricultural Implement Manufacturing',
            'Industrial Machinery Manufacturing',
            'Metalworking Machinery Manufacturing',
            'Other General Purpose Machinery Manufacturing'
        ]
    },
    # Include other mappings as required
}


In [22]:
import sys
sys.path.append('/content/data-driven-modelling/notebooks')

from func import DataWrangling

In [23]:
wrangler = DataWrangling()

In [24]:
df_pattern = pattern_data

In [25]:
processed_df = wrangler.process_naics_data(df_pattern, naics_expanding)

In [ ]:
# from main import DataWrangling  # Assuming the class is in a file named main.py

# # Step 1: Load data
# # gdp_data = filtered_gdp
# # occupation_data = filtered_occupation

# # # Step 2: Expand NAICS codes
# pattern_data_expanded = DataWrangling.expand_naics_details(pattern_data, naics_expanding)

# # # Step 3: Merge GDP and Pattern data
# # merged_data = DataWrangling.merge_and_filter_datasets(gdp_data, pattern_data_expanded)

# # # Step 4: Aggregate data
# # aggregated_data = DataProcessing.aggregate_data(merged_data)

# # # Step 5: Process Occupation data
# # filtered_occ_codes = ['47-1011', '49-1011']  # Replace with your prioritized codes
# # processed_occupation_data = DataProcessing.process_occupation_data(occupation_data, filtered_occ_codes)

# # # Step 6: Combine datasets
# # final_dataset = DataProcessing.final_merge_and_aggregation(aggregated_data, processed_occupation_data)

# # # Step 7: Save final dataset
# # final_dataset.to_pickle('processed_data/pkl/final_dataset.pkl')



In [ ]:
processed_df.head()

,FIPS,naics,DESCRIPTION,emp,ap,est
0,1001,1133,Logging,68,4563,7
1,1001,2123,Nonmetallic Mineral Mining and Quarrying,87,5144,6
2,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4
3,1001,2362,Nonresidential Building Construction,69,4184,7
4,1001,2371,Utility System Construction,65,4574,4


# Tool consumption

Here, the individual industries must be evaluated by **domain knowledge** with regard to their potential tool consumption.<br>
We will insert random values between 0 and 10 as an example

**We gonna do that 2 times**

*   With the Industry Pattern Dataset
*   With the Occupation Dataset

<br>

---




**1.   FIRST Start with the Industries Pattern **




In [26]:
# Generate random values for each NAICS and assign them to each entry in the group
random_values = {naics: np.random.randint(0, 11) for naics in processed_df['naics'].unique()}
processed_df['tool_consumption'] = processed_df['naics'].map(random_values)
processed_df.head(3)

,FIPS,naics,DESCRIPTION,emp,ap,est,tool_consumption
0,1001,1133,Logging,68,4563,7,3
1,1001,2123,Nonmetallic Mineral Mining and Quarrying,87,5144,6,7
2,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,5


In [27]:
# Generate random values for each NAICS and assign them to each entry in the group
random_values = {naics: np.random.randint(0, 11) for naics in filtered_occupation['naics'].unique()}
filtered_occupation['tool_consumption_occ'] = filtered_occupation['naics'].map(random_values)
filtered_occupation.head(3)

,FIPS,naics,NAICS_TITLE,OCC_CODE,OCC_TITLE,emp_occupation,tool_consumption_occ
0,12999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",20639.514235,5
1,6999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",9414.167765,5
2,36999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",8332.850279,5


## Industries Before Ranking

In [28]:
pattern_result = processed_df.groupby(['naics', 'DESCRIPTION', 'tool_consumption']).agg(
    emp_sum=('emp', 'sum'),            # Sum of values in the 'emp' column
    unique_FIPS=('FIPS', 'nunique')    # Count of unique values in the 'FIPS' column
).sort_values(by='emp_sum', ascending=False).reset_index()

# Display the first 20 rows
pattern_result.head(20)

,naics,DESCRIPTION,tool_consumption,emp_sum,unique_FIPS
0,5613,Employment Services,1,7061425,1306
1,2382,Building Equipment Contractors,8,2171001,2756
2,5617,Services to Buildings and Dwellings,10,2004960,2530
3,5413,"Architectural, Engineering, and Related Services",8,1579218,1924
4,4411,Automobile Dealers,7,1245435,1953
5,7139,Other Amusement and Recreation Industries,4,1092894,2182
6,8111,Automotive Repair and Maintenance,0,902870,2725
7,2381,"Foundation, Structure, and Building Exterior C...",7,901696,2313
8,2383,Building Finishing Contractors,2,811812,2040
9,4238,"Machinery, Equipment, and Supplies Merchant Wh...",1,739134,1947


In [29]:
# Create a copy of the DataFrame
pattern_result_copy = pattern_result.copy()

# Calculate rank for all columns starting from 'tool_consumption', with equal values having the same rank
for column in pattern_result_copy.columns[pattern_result_copy.columns.get_loc('tool_consumption'):]:
    rank_column_name = f'rank_{column}'
    pattern_result_copy[rank_column_name] = pattern_result_copy[column].rank(method='min', ascending=False).astype(int)

# Display the first few rows of the copied data
pattern_result_copy.head()

,naics,DESCRIPTION,tool_consumption,emp_sum,unique_FIPS,rank_tool_consumption,rank_emp_sum,rank_unique_FIPS
0,5613,Employment Services,1,7061425,1306,57,1,14
1,2382,Building Equipment Contractors,8,2171001,2756,11,2,1
2,5617,Services to Buildings and Dwellings,10,2004960,2530,1,3,3
3,5413,"Architectural, Engineering, and Related Services",8,1579218,1924,11,4,10
4,4411,Automobile Dealers,7,1245435,1953,17,5,8


## Industries After Ranking

Adding Weights and for sure the highest weight will go for the **Rank Tool Consumption**

In [30]:
weights = {
    'rank_tool_consumption': 0.5,   # Weight for the ranking column 'tool_consumption'
    'rank_emp_sum': 0.3,            # Weight for the ranking column 'emp_sum'
    'rank_unique_FIPS': 0.2,         # Weight for the ranking column 'unique_FIPS'
}

In [31]:
# calculate the weighted sum
pattern_result_copy['Weighted_Sum'] = (pattern_result_copy['rank_tool_consumption'] * weights['rank_tool_consumption'] +
                          pattern_result_copy['rank_emp_sum'] * weights['rank_emp_sum'] +
                          pattern_result_copy['rank_unique_FIPS'] * weights['rank_unique_FIPS']
                          ) # Display the first few rows
pattern_sorted = pattern_result_copy.sort_values(by='Weighted_Sum', ascending=True)

In [32]:
#Final Sorted Top Industries to be used later for filtering industies
pattern_sorted.head(20)

,naics,DESCRIPTION,tool_consumption,emp_sum,unique_FIPS,rank_tool_consumption,rank_emp_sum,rank_unique_FIPS,Weighted_Sum
2,5617,Services to Buildings and Dwellings,10,2004960,2530,1,3,3,2.0
1,2382,Building Equipment Contractors,8,2171001,2756,11,2,1,6.3
3,5413,"Architectural, Engineering, and Related Services",8,1579218,1924,11,4,10,8.7
4,4411,Automobile Dealers,7,1245435,1953,17,5,8,11.6
7,2381,"Foundation, Structure, and Building Exterior C...",7,901696,2313,17,8,5,11.9
10,2362,Nonresidential Building Construction,7,657152,1687,17,11,11,14.0
32,3219,Other Wood Product Manufacturing,10,214762,967,1,33,21,14.6
39,5321,Automotive Equipment Rental and Leasing,10,137245,784,1,40,26,17.7
20,3231,Printing and Related Support Activities,7,356146,1116,17,21,18,18.4
14,3330,"3331, 3332, 3334, 3339",7,488895,699,17,15,28,18.6


In [33]:

top20_industries = pattern_sorted[:20]['naics'].tolist()
top20_industries

['5617',
 '2382',
 '5413',
 '4411',
 '2381',
 '2362',
 '3219',
 '5321',
 '3231',
 '3330',
 '3320',
 '7139',
 '3399',
 '4412',
 '2211',
 '3320',
 '3344',
 '3327',
 '2212',
 '3119']

In [47]:
# Creating DataFrame with a specified column name
df_Top_Industry = pd.DataFrame(top20_industries, columns=['naics'])

print(df_Top_Industry)

   naics
0   5617
1   2382
2   5413
3   4411
4   2381
5   2362
6   3219
7   5321
8   3231
9   3330
10  3320
11  7139
12  3399
13  4412
14  2211
15  3320
16  3344
17  3327
18  2212
19  3119


**The following industries needs to be cleaned**

*   3320A1
*   3330A1

Will need to use the link here https://www.bls.gov/oes/2023/may/naics4_3320A2.htm <br> to Check what those 6 dig codes represesnts actually in 4 gid







---





**2.   SECOND Move to the Occupation **




In [34]:
filtered_occupation.head()


,FIPS,naics,NAICS_TITLE,OCC_CODE,OCC_TITLE,emp_occupation,tool_consumption_occ
0,12999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",20639.514235,5
1,6999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",9414.167765,5
2,36999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",8332.850279,5
3,6037,8111,Automotive Repair and Maintenance,49-3023,Automotive Service Technicians and Mechanics,5913.423292,6
4,48999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",5770.378034,5


In [35]:
occupation_result = filtered_occupation.groupby(['OCC_CODE','OCC_TITLE', 'tool_consumption_occ']).agg(
    emp_occ_sum=('emp_occupation', 'sum'),             # Sum of values in the 'emp' column
    unique_FIPS=('FIPS', 'nunique'),    # Count of unique values in the 'FIPS' column
).sort_values(by='emp_occ_sum', ascending=False).reset_index()

# Display the first 20 rows
occupation_result.head(20)

,OCC_CODE,OCC_TITLE,tool_consumption_occ,emp_occ_sum,unique_FIPS
0,49-3023,Automotive Service Technicians and Mechanics,6,256892.826518,2765
1,49-9071,"Maintenance and Repair Workers, General",5,131698.939688,1713
2,51-4041,Machinists,5,123371.831501,1606
3,51-4121,"Welders, Cutters, Solderers, and Brazers",7,106829.485387,1852
4,49-3021,Automotive Body and Related Repairers,6,104132.142021,2725
5,51-4072,"Molding, Coremaking, and Casting Machine Sette...",10,95590.294695,819
6,49-9071,"Maintenance and Repair Workers, General",7,74752.691258,1860
7,51-4031,"Cutting, Punching, and Press Machine Setters, ...",4,71186.656292,777
8,51-4121,"Welders, Cutters, Solderers, and Brazers",2,70142.387247,1850
9,17-2141,Mechanical Engineers,8,69636.370405,2140


In [36]:
# Create a copy of the DataFrame
occupation_result_copy = occupation_result.copy()

# Calculate rank for all columns starting from 'tool_consumption', with equal values having the same rank
for column in occupation_result_copy.columns[occupation_result_copy.columns.get_loc('tool_consumption_occ'):]:
    rank_column_name = f'rank_{column}'
    occupation_result_copy[rank_column_name] = occupation_result_copy[column].rank(method='min', ascending=False).astype(int)

# Display the first few rows of the copied data
occupation_result_copy.head()

,OCC_CODE,OCC_TITLE,tool_consumption_occ,emp_occ_sum,unique_FIPS,rank_tool_consumption_occ,rank_emp_occ_sum,rank_unique_FIPS
0,49-3023,Automotive Service Technicians and Mechanics,6,256892.826518,2765,227,1,8
1,49-9071,"Maintenance and Repair Workers, General",5,131698.939688,1713,288,2,119
2,51-4041,Machinists,5,123371.831501,1606,288,3,151
3,51-4121,"Welders, Cutters, Solderers, and Brazers",7,106829.485387,1852,156,4,81
4,49-3021,Automotive Body and Related Repairers,6,104132.142021,2725,227,5,14


In [37]:
weights = {
    'rank_tool_consumption_occ': 0.5,   # Weight for the ranking column 'tool_consumption'
    'rank_emp_occ_sum': 0.3,            # Weight for the ranking column 'emp_sum'
    'rank_unique_FIPS': 0.2,         # Weight for the ranking column 'unique_FIPS'
}


In [38]:
# calculate the weighted sum
occupation_result_copy['Weighted_Sum_Occ'] = (occupation_result_copy['rank_tool_consumption_occ'] * weights['rank_tool_consumption_occ'] +
                          occupation_result_copy['rank_emp_occ_sum'] * weights['rank_emp_occ_sum'] +
                          occupation_result_copy['rank_unique_FIPS'] * weights['rank_unique_FIPS']
                          ) # Display the first few rows
occupation_sorted = occupation_result_copy.sort_values(by='Weighted_Sum_Occ', ascending=True)

In [40]:
occupation_sorted.head(20)

,OCC_CODE,OCC_TITLE,tool_consumption_occ,emp_occ_sum,unique_FIPS,rank_tool_consumption_occ,rank_emp_occ_sum,rank_unique_FIPS,Weighted_Sum_Occ
12,47-2211,Sheet Metal Workers,9,62955.769115,2756,56,13,9,33.7
20,51-4041,Machinists,9,38146.999138,2424,56,21,26,39.5
45,49-9071,"Maintenance and Repair Workers, General",9,24172.079988,2787,56,46,1,42.0
53,17-2141,Mechanical Engineers,9,22150.664108,2779,56,54,7,45.6
44,17-2112,Industrial Engineers,9,25045.487980,2444,56,45,24,46.3
62,51-4121,"Welders, Cutters, Solderers, and Brazers",9,19302.379770,2787,56,63,1,47.1
77,49-9041,Industrial Machinery Mechanics,9,14796.487712,2786,56,78,3,52.0
35,49-9041,Industrial Machinery Mechanics,10,30391.710176,1249,1,36,235,58.3
34,49-9071,"Maintenance and Repair Workers, General",10,30549.813028,1248,1,35,237,58.4
29,17-2112,Industrial Engineers,10,32469.562302,1220,1,30,249,59.3


In [43]:

top20_occ = occupation_sorted[:20]['OCC_CODE'].tolist()


In [46]:
top20_occ

['47-2211',
 '51-4041',
 '49-9071',
 '17-2141',
 '17-2112',
 '51-4121',
 '49-9041',
 '49-9041',
 '49-9071',
 '17-2112',
 '49-9044',
 '11-3051',
 '17-2141',
 '11-3051',
 '17-2141',
 '17-2112',
 '51-4072',
 '51-4041',
 '51-4021',
 '51-4121']

In [48]:
# Creating DataFrame with a specified column name
df_Top_OCC = pd.DataFrame(top20_occ, columns=['OCC_CODE'])

print(df_Top_OCC)

   OCC_CODE
0   47-2211
1   51-4041
2   49-9071
3   17-2141
4   17-2112
5   51-4121
6   49-9041
7   49-9041
8   49-9071
9   17-2112
10  49-9044
11  11-3051
12  17-2141
13  11-3051
14  17-2141
15  17-2112
16  51-4072
17  51-4041
18  51-4021
19  51-4121



### NOW, gonna filter the dataframe of the occupations and the patterna and merge all to have one master dataframe

In [55]:
filtered_pattern_data = processed_df[processed_df['naics'].isin(top20_industries)]
filtered_occupation_data = df_filtered_occupation[df_filtered_occupation['OCC_CODE'].isin(top20_occ)]

In [59]:
filtered_occupation_data

,FIPS,naics,NAICS_TITLE,OCC_CODE,OCC_TITLE,emp_occupation
0,12999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",20639.514235
1,6999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",9414.167765
2,36999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",8332.850279
4,48999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",5770.378034
5,18039,3362,Motor Vehicle Body and Trailer Manufacturing,51-4121,"Welders, Cutters, Solderers, and Brazers",5303.207380
...,...,...,...,...,...,...
1019541,15001,2383,Building Finishing Contractors,49-9044,Millwrights,0.000000
1019542,15001,3119,Other Food Manufacturing,49-9044,Millwrights,0.000000
1019543,15001,3219,Other Wood Product Manufacturing,49-9044,Millwrights,0.000000
1019544,15001,3370A1,"3371, 3372",49-9044,Millwrights,0.000000


In [60]:
# Merge Pattern and Occupation Data
merged_data = pd.merge(
    filtered_pattern_data,
    filtered_occupation_data,
    on='FIPS',
    how='inner'
)

# # Merge with GDP Data
# final_merged_data = pd.merge(
#     merged_data,
#     gdp_data[['FIPS', '2022']],  # Include only relevant columns from GDP data
#     on='FIPS',
#     how='inner'
# )


In [61]:
merged_data

,FIPS,naics_x,DESCRIPTION,emp,ap,est,tool_consumption,naics_y,NAICS_TITLE,OCC_CODE,OCC_TITLE,emp_occupation
0,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,5,2211,"Electric Power Generation, Transmission and Di...",49-9041,Industrial Machinery Mechanics,6.767063
1,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,5,2123,Nonmetallic Mineral Mining and Quarrying,49-9041,Industrial Machinery Mechanics,5.650169
2,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,5,2382,Building Equipment Contractors,47-2211,Sheet Metal Workers,5.299248
3,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,5,2123,Nonmetallic Mineral Mining and Quarrying,49-9071,"Maintenance and Repair Workers, General",2.639348
4,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,5,5617,Services to Buildings and Dwellings,49-9071,"Maintenance and Repair Workers, General",2.469067
...,...,...,...,...,...,...,...,...,...,...,...,...
4667897,56999,5413,"Architectural, Engineering, and Related Services",126,13017,9,8,5613,Employment Services,47-2211,Sheet Metal Workers,0.034154
4667898,56999,5413,"Architectural, Engineering, and Related Services",126,13017,9,8,5613,Employment Services,49-9044,Millwrights,0.029851
4667899,56999,5413,"Architectural, Engineering, and Related Services",126,13017,9,8,5613,Employment Services,11-3051,Industrial Production Managers,0.022005
4667900,56999,5413,"Architectural, Engineering, and Related Services",126,13017,9,8,2211,"Electric Power Generation, Transmission and Di...",17-2141,Mechanical Engineers,0.012669


In [62]:
merge_columns_to_drop_gdp = [
    'tool_consumption', 'naics_y', 'NAICS_TITLE'
]

# Revomve not needed columns
filtered_merge = merged_data.drop(columns=merge_columns_to_drop_gdp, errors='ignore')

# Show the rest
print(filtered_merge.columns)

Index(['FIPS', 'naics_x', 'DESCRIPTION', 'emp', 'ap', 'est', 'OCC_CODE',
       'OCC_TITLE', 'emp_occupation'],
      dtype='object')


In [63]:
filtered_merge

,FIPS,naics_x,DESCRIPTION,emp,ap,est,OCC_CODE,OCC_TITLE,emp_occupation
0,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,49-9041,Industrial Machinery Mechanics,6.767063
1,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,49-9041,Industrial Machinery Mechanics,5.650169
2,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,47-2211,Sheet Metal Workers,5.299248
3,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,49-9071,"Maintenance and Repair Workers, General",2.639348
4,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,49-9071,"Maintenance and Repair Workers, General",2.469067
...,...,...,...,...,...,...,...,...,...
4667897,56999,5413,"Architectural, Engineering, and Related Services",126,13017,9,47-2211,Sheet Metal Workers,0.034154
4667898,56999,5413,"Architectural, Engineering, and Related Services",126,13017,9,49-9044,Millwrights,0.029851
4667899,56999,5413,"Architectural, Engineering, and Related Services",126,13017,9,11-3051,Industrial Production Managers,0.022005
4667900,56999,5413,"Architectural, Engineering, and Related Services",126,13017,9,17-2141,Mechanical Engineers,0.012669


**Now, we will filter the Occupations for the ones that is gonna use our products**

*   Assume some occupations that are important like
  1.   '51-4121' > Welders, Cutters, Solderers, and Brazers
  2.   '47-2221' > Structural Iron and Steel Workers
  3.   '51-2041' > Structural Metal Fabricators and Fitters
  4.   '49-3021' > Automotive Body and Related Repairers
  5.   '51-4041' > Machinists
  6.   '49-9041' > Industrial Machinery Mechanics
  7.   '49-9071' > Maintenance and Repair Workers, General
  8.   '51-4081' > Multiple Machine Tool Setters, Operators, and Tenders, Metal and Plastic
  9.   '47-2211' > Sheet Metal Workers
  10.  '49-3031' > Bus and Truck Mechanics and Diesel Engine Specialists
  11.   '51-4033' > Grinding, Lapping, Polishing, and Buffing Machine Tool Setters, Operators, and Tenders, Metal and Plastic
  12.   '49-3023' > Automotive Service Technicians and Mechanics
  13.   '47-2011' > Boilermakers
  14.   '51-4122' > Welding, Soldering, and Brazing Machine Setters, Operators, and Tenders
  15.   '51-9021' > Crushing, Grinding, and Polishing Machine Setters, Operators, and Tenders
  16.   '51-4031' > Cutting, Punching, and Press Machine Setters, Operators, and Tenders, Metal and Plastic
  17.   '49-3011' > Aircraft Mechanics and Service Technicians
  18.   '51-4111' > Tool and Die Makers
  19.   '51-9032' > Cutting and Slicing Machine Setters, Operators, and Tenders
  20.   '49-9043' > Maintenance Workers, Machinery


# Feature overview

# Feature selection

# Feature construction

# feature_df

# Scaling

# Dimension reduction

# **Loading Map & Ploting it**

After looking in the County_Data

*   The column named "GEOID", should be remaned to "FIPS"
*   The FIPS column datatype should be changed to int


In [ ]:
county_data.rename(columns={'GEOID': 'FIPS'}, inplace=True)

In [ ]:
county_data = county_data.astype({"FIPS": int})

We will load the map csv amd plot it

Then apply then will merge it to the other dateframes

In [ ]:
# دمج البيانات الجغرافية مع الـ GDP
merged_data = pd.merge(county_data, gdp_data, on='GEOID', how='inner')

# دمج البيانات الصناعية
merged_data = pd.merge(merged_data, pattern_data, on='FIPS', how='inner')

# دمج بيانات الوظائف
merged_data = pd.merge(merged_data, occupation_data, on='FIPS', how='inner')

# دمج بيانات الولايات
merged_data = pd.merge(merged_data, state_data, on='STATEFP', how='inner')


NameError: name 'gdp_data' is not defined

## PCA

In [ ]:
# This code extracts the top 10 and the next 10 'OCC_CODE' values from the weighted ranked occupation data and stores them
# in two separate lists: 'occ_top10' and 'occ_top10_20'.
# It also defines two lists of NAICS codes: 'naics_top6' for the top 6 NAICS codes and 'naics_top_metall' for NAICS codes
# related to the metallurgical industry.
# # Finally, it prints the contents of the 'occ_top10', 'occ_top10_20', 'naics_top6', and 'naics_top_metall' lists.

occ_top10 = ranked_occupation_data_weighted['OCC_CODE'][:10].tolist()
occ_top20 = ['51-4121', '47-2221', '51-2041', '49-3021', '51-4041', '49-9041', '49-9071', '51-4081', '47-2211', '49-3031']
# #occ_top10_20 = ranked_occupation_data_weighted['OCC_CODE'][10:20].tolist()
# occ_top10_20 = ['51-4033', '49-3023', '47-2011', '51-4122', '51-9021', '51-4031', '49-3011', '51-4111', '51-9032', '49-9043']
# naics_top6 = ['2382','8111','3320A2','3330A1','3327','3363']
# naics_top_metall = ['3320A1','3335','3364','3362','3315','3366','2379','3336','3311','3314']
# print(occ_top10)
# print(occ_top10_20)
# print(naics_top6)
# print(naics_top_metall)